In [4]:
pip install pandas

In [5]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [6]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [7]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


In [8]:
inputs  = pd.get_dummies(inputs,dummy_na =True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


In [10]:
import torch

X,y = torch.tensor(inputs.values),torch.tensor(outputs.values)
X,y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

###  1. 文件与数据创建
目录创建：使用 os.makedirs() 创建数据存储目录（如 data 文件夹），可通过参数避免目录已存在时的报错。
CSV 文件生成：手动构造或批量生成含特征（如房间数、小巷类型）和目标变量（如房价）的 CSV 数据，需注意缺失值用 NA 标记。
###  2. 数据读取与拆分
CSV 读取：用 pandas.read_csv() 加载表格数据，返回 DataFrame 格式，便于后续数据操作。
输入输出分离：根据建模需求拆分数据 —— 特征列（如 NumRooms、Alley）作为模型输入（X），目标列（如 Price）作为预测输出（y）。
###  3. 缺失值处理（数值型特征）
均值填充：针对数值型缺失特征（如 NumRooms），用 DataFrame.fillna(DataFrame.mean()) 填充，核心是用该列非缺失值的均值替代缺失值，适用于缺失值较少、数据分布较均匀的场景。
###  4. 分类特征编码
独热编码：针对离散分类特征（如 Alley，取值为 Pave 或缺失），用 pandas.get_dummies() 实现编码：
将每个类别映射为一个二进制列（如 Alley_Pave、Alley_nan）；
缺失值会被自动视为一个独立类别，避免信息丢失。
###  5. 数据类型适配（PyTorch）
张量转换：用 torch.tensor() 将处理后的 DataFrame（输入 X、输出 y）转为 PyTorch 张量（Tensor），张量是 PyTorch 模型的核心输入格式，需确保数据类型为数值型（如 float）。
###  6. 关键数据预处理原则
数值型特征：优先用均值 / 中位数填充缺失值，避免直接删除数据；
分类特征：低基数（类别少）特征用独热编码，高基数特征需其他编码方式（如嵌入编码）；
数据一致性：预处理后需确保输入和输出的维度匹配，无缺失值残留，满足模型输入要求。